In [1]:
!pip install -e ..

Obtaining file:///root
  Preparing metadata (setup.py) ... done
  Attempting uninstall: cleangpt
    Found existing installation: cleangpt 0.1.0
    Uninstalling cleangpt-0.1.0:
      Successfully uninstalled cleangpt-0.1.0
  Running setup.py develop for cleangpt


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import subprocess
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
import torch
from IPython.display import display, HTML, clear_output
from tqdm.notebook import tqdm
from cleangpt.gpt import make as make_gpt
from cleangpt.exp.data import make_loader, tokenize, DATA_PATH
from cleangpt.exp.train import Trainer

In [4]:
# assert torch.cuda.is_available() is False
assert torch.cuda.is_available()

def get_gpu_utilization(gpu_id):
    try:
      result = subprocess.run([
          'nvidia-smi', 
          '--query-gpu=utilization.gpu',
          '--format=csv,nounits,noheader',
          '-i', str(gpu_id)
        ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    except Exception as e:
      print(f"Error getting GPU utilization for GPU {gpu_id}: {e}")
    utilization = float(result.stdout.strip())
    return utilization

def get_device():
  num_gpus = torch.cuda.device_count()
  if num_gpus == 0:
    return "cpu"
  gpu_id = min(range(num_gpus), key=get_gpu_utilization)
  return f"cuda:{gpu_id}"

In [5]:
device = get_device()
device

'cuda:0'

In [6]:
OUTPUT_PATH = os.path.join(DATA_PATH, "generated")

if not os.path.isdir(OUTPUT_PATH):
  os.makedirs(OUTPUT_PATH)

In [7]:
model = make_gpt(vocab_size=656, seqlen=128, config_name="gpt2").to(device)
trainer = Trainer(model)

In [8]:
dloader = make_loader(batch_size=64, nmerged=400, num_workers=0)

In [9]:
display(HTML(dloader.dataset.decode(dloader.dataset.tokens)))

1.,A cognitive state is epistemically independent if it possesses its epistemic status independently of its being inferred or inferrable from some other cognitive state.
,[Definition of epistemic independence]
2.,A cognitive state is epistemically efficacious — is capable of epistemically supporting other cognitive states — if the epistemic status of those other states can be validly inferred (formally or materially) from its epistemic status.
,[Definition of epistemic efficacy]
3,"The doctrine of the given is that any empirical knowledge that p requires some (or is itself) basic, that is, epistemically independent, knowledge (that g, h, i, …) which is epistemically efficacious with respect to p."
,[Definition of doctrine of the given]
4.,Inferential relations are always between items with propositional form.
,[By the nature of inference]
5.,"Therefore, non-propositional items (such as sense data) are epistemically inefficacious and cannot serve as what is given."
,[From 2 and 4]
6.,"No inferentially acquired, propositionally structured mental state is epistemically independent."


In [10]:
nepochs = 100

In [11]:
def generate(epoch, model, period=20,
             inputs=dloader.dataset[0][0][None].to(device),
             num_new_tokens=len(dloader.dataset.tokens) - len(dloader.dataset[0][0]),
             decode=dloader.dataset.decode,
             output_path=os.path.join(OUTPUT_PATH, dloader.dataset.filename),
             **kwargs):
  if epoch < period: 
    return
  if epoch > period and epoch % period != 0:
    with open(output_path, 'r', encoding="utf8") as infile:
      display(HTML(infile.read()))
    return
  model.eval()
  outputs = model.generate(inputs, num_new_tokens, **kwargs)
  text = decode(outputs[0].tolist())
  with open(output_path, 'w', encoding="utf8") as outfile:
    outfile.write(text)
  display(HTML(text))
  model.train()

In [12]:
def update(source, handle, x, y):
  source.stream(dict(x=[x], y=[y]))
  return push_notebook(handle=handle)

In [13]:
output_notebook()
source = ColumnDataSource(data=dict(x=[], y=[]))
plot = figure(title="Loss", y_axis_type="log")
line = plot.line('x', 'y', source=source)
handle = show(plot, notebook_handle=True)

with tqdm(total=nepochs) as pbar:
  for n in range(nepochs):
    for i, (inputs, targets, loss) in enumerate(trainer.iter_epoch(dloader)):
      if max(i, n) == 0 or i in {j * len(dloader) // 3 for j in range(1, 3)}:
        handle = update(source, handle, n * len(dloader) + i, float(loss))
    clear_output(True)
    pbar.update(1)
    display(pbar.container)
    handle = show(plot, notebook_handle=True)
    generate(n, model, sample=True, topk=10)

100%|##########| 100/100 [12:42:33<00:00, 348.18s/it]

1.,A cognitive state is epistemically independent if it possesses its epistemic status independently of its being inferred or inferrable from some other cognitive state.
,[Definition of epistemic independence]
2.,A cognitive state is epistemically efficacious — is capable of epistemically supporting other cognitive states — if the epistemic status of those other states can be validly inferred (formally or materially) from its epistemic status.
,[Definition of epistemic efficacy]
3,"The doctrine of the given is that any empirical knowledge that p requires some (or is itself) basic, that is, epistemically independent, knowledge (that g, h, i, …) which is epistemically efficacious with respect to p."
,[Definition of doctrine of the given]
4.,Inferential relations are always between items with propositional form.
,[By the nature of inference]
5.,"Therefore, non-propositional items (such as sense data) are epistemically inefficacious and cannot serve as what is given."
,[From 2 and 4]
6.,"No inferentially acquired, propositionally structured mental state is epistemically independent."
